In [8]:
from tensorflow import keras

# データの拡張
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### ImageNetからのトレーニング済みモデルのダウンロード
[Keras Documentation](https://keras.io/ja/applications/#vgg16)  
[Keras：VGG16、VGG19とかってなんだっけ？？](https://qiita.com/MuAuan/items/86a56637a1ebf455e180)  

In [2]:
# 59MBのダウンロードが必要
pre_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [3]:
pre_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### ベースモデルの凍結
[学習済みCNNモデルの利用方法まとめ【ディープラーニング】](https://sinyblog.com/deaplearning/pre_train_model/)  
[TensorFlow, Kerasでレイヤー、モデルのtrainable属性を設定（Freeze / Unfreeze）](https://note.nkmk.me/python-tensorflow-keras-trainable-freeze-unfreeze/)

In [4]:
# false で凍結
pre_model.trainable = False

### 新しいレイヤーの追加
[Keras Documentation](https://keras.io/ja/layers/pooling/)  
[全体平均プーリング（1/5）背景技術](https://benrishi-ai.com/gapooling01/)  
[Global Average Pooling（GAP）を理解してみる](https://qiita.com/mine820/items/1e49bca6d215ce88594a)
[Keras Documentation Dense](https://keras.io/ja/layers/core/)  
[Keras Documentation 活性化関数の使い方](https://keras.io/ja/activations/)

In [5]:
### keras.Input()とは？
inputs = keras.Input(shape=(224, 224, 3))

x = image_model(inputs, training=False)

# 空間データのグローバルな平均プーリング演算
# 空間データとは「モノの位置情報と意味情報（状態、形状、サイズなど）の双方を要素として持つデータ」であり、
# 具体的には「車両が交差点を5km/hで走行している」というようなものを指します。
x = keras.layers.GlobalAveragePooling2D()(x)

# Dense 密集
# 通常の全結合ニューラルネットワークレイヤー

# units：出力空間の次元数
units = 6
# activation は 活性関数
outputs = keras.layers.Dense(units, activation = 'softmax')(x)

# 
revised_model = keras.Model(inputs, outputs)

In [ ]:
revised_model.summary()

### モデルのコンパイル
[loss 損失関数](https://keras.io/api/losses/) モデルがトレーニング中に最小化しようとする量を計算すること  
損失関数は予測と実際の値のズレの大きさを表す関数でモデルの予測精度を評価します。損失関数の値が小さければより正確なモデル  

交差エントロピー誤差
この損失関数は分類問題に用います。以下の数式で表されるのが交差エントロピー誤差  

$$
    E=-\logyk
$$

[機械学習における損失関数の役割や種類](https://ai-trend.jp/basic-study/neural-network/nn_loss_function/#:~:text=%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%82%92%E3%81%AF%E3%81%98%E3%82%81%E3%81%A8,%E3%81%AB%E8%80%83%E3%81%88%E3%81%A6%E3%81%84%E3%81%8D%E3%81%BE%E3%81%97%E3%82%87%E3%81%86%E3%80%82)

merics 評価関数 訓練時とテスト時にモデルにより評価される評価関数のリスト

In [6]:
model.compile(loss = keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics = [keras.metrics.categorical_accuracy])

### データの拡張

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True # we don't expect Bo to be upside-down so we will not flip vertically
)

### データセットの読み込み

In [ ]:
train_data_path = 
test_data_path = 

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory(train_data_path,
                                       target_size=(244, 244), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate test dataset
test_it = datagen.flow_from_directory(test_data_path,
                                      target_size=(244, 244), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

### モデルのトレーニング

In [ ]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=20)

### モデルの凍結を解除してファインチューニング

In [ ]:
# 凍結の解除
base_model.trainable = True

In [ ]:
# コンパイル
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.BinaryCrossentropy(from_logits=True) , metrics =  [keras.metrics.categorical_accuracy])

In [ ]:
# 学習
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=10)

### モデルの評価

In [ ]:
model.evaluate(test_it, steps=test_it.samples/test_it.batch_size)

### 分類結果の確認

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

### 分類結果の確認

In [ ]:
temp1 = make_predictions('fruits/test/freshapples/Screen Shot 2018-06-08 at 5.27.54 PM.png')
np.argmax(temp1)

In [ ]:
# 判定結果を表示する関数
def tasting_fruits(image_path):
    preds = make_predictions(image_path)
    if np.argmax(preds)>=4:
        print("It's ratten! Not yammy! Boo!")
    else:
        print("It's fresh! Yammy! Yes!")